# Measuring peformance in OpenCL applications

Having an understanding of how well OpenCL applications perform is a vital part of the development process. The two main tools, **profiling** and **tracing** collect information about how well an application is performing. **Profiling** is the statistical collection of the cumulative time that threads spend in each program component. **Tracing** is the collection of both **when** and **for how long** threads spend in each application component. While it is true that many vendors have largely abandoned their OpenCL performance measurement tools, the OpenCL standard itself provides a profiling interface and there are still a few open-source and commercial tools available.

## Event based timing

Events in OpenCL are used with streams to check the progress of work that has been submitted and establish dependencies between workflows. They can also be used to time the execution of work, such as kernels and memory copies. Here is how Events fit into the picture of an OpenCL application.

<figure style="margin-left:auto; margin-right:auto; width:70%;">
    <img style="vertical-
                align:middle" src="../images/opencl_components.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Components of an OpenCL application. Events are associated with command queues, and provide a way to time the duration of work in a command queue. </figcaption>
</figure>

Command queues have the ability to capture timing information for work submitted. In order to time commands submitted to an OpenCL command queue we enable a profiling flag called **CL_QUEUE_PROFILING_ENABLE** at command queue creation. Then time elapsed may be extracted directly from an event. In the code [mat_mult_profiling.cpp](mat_mult_profiling.cpp) we set the profiling flag to CL_TRUE.

```C++
    // mat_mult_profiling.cpp source

    // Do we enable profiling?
    cl_bool profiling = CL_TRUE;
```

Then from within **h_create_command_queues** in <a href="../include/cl_helper.hpp">cl_helper.hpp</a>, the profiling flag **CL_QUEUE_PROFILING_ENABLE** is incorporated into the command queue properties and passed to either [clCreateCommandQueue](https://www.khronos.org/registry/OpenCL/sdk/3.0/docs/man/html/clCreateCommandQueue.html) from the OpenCL 1.2 API or [clCreateCommandQueueWithProperties](https://www.khronos.org/registry/OpenCL/sdk/3.0/docs/man/html/clCreateCommandQueueWithProperties.html) from the OpenCL 2.0 API.

```C++
    // cl_helper.hpp source

#ifdef CL_VERSION_2_0
        // Check to see what version the device supports
        cl_float device_ver = h_get_device_ver(device);

        if (device_ver>=2.0) {
            // Create the command queue the OpenCL 2.0 way
            cl_queue_properties queue2_props[] = {
                CL_QUEUE_PROPERTIES, queue_properties,
                0
            };
            // Use the OpenCL 2.0 API    
            command_queues[n] = clCreateCommandQueueWithProperties(
                context,
                device,
                queue2_props,
                &errcode    
            );
            h_errchk(errcode, "Creating an OpenCL 2.0 command queue with properties");  

        } else 
#endif
        { 
            // Create the command queue the OpenCL 1.2 way
            command_queues[n] = clCreateCommandQueue(
                context,
                device,
                queue_properties,
                &errcode    
            );

            h_errchk(errcode, "Creating an OpenCL 1.2 command queue");  
        }
```

The function [clGetEventProfilingInfo](https://www.khronos.org/registry/OpenCL/sdk/3.0/docs/man/html/clGetEventProfilingInfo.html) extracts information such as start and end walltimes (in nanoseconds) for an OpenCL event associated with a queued command. We use the helper function **h_get_event_time_ms** in <a href="../include/cl_helper.hpp">cl_helper.hpp</a> to extract the elapsed time.

```C++

// cl_helper.hpp source

cl_double h_get_event_time_ms(
        cl_event *event, 
        const char* message, 
        size_t* nbytes) {
    
    // Make sure the event has finished
    h_errchk(clWaitForEvents(1, event), message);
    
    // Start and end times
    cl_ulong t1, t2;
        
    // Fetch the start and end times in nanoseconds
    h_errchk(
        clGetEventProfilingInfo(
            *event,
            CL_PROFILING_COMMAND_START,
            sizeof(cl_ulong),
            &t1,
            NULL
        ),
        "Fetching start time for event"
    );

    h_errchk(
        clGetEventProfilingInfo(
            *event,
            CL_PROFILING_COMMAND_END,
            sizeof(cl_ulong),
            &t2,
            NULL
        ),
        "Fetching end time for event"
    );
    
    // Convert the time into milliseconds
    cl_double elapsed = (cl_double)(t2-t1)*(cl_double)1.0e-6;
        
    // Print the timing message if necessary
    if (strlen(message)>0) {
        std::printf("Time for event \"%s\": %.3f ms", message, elapsed);
        
        // Print transfer rate if nbytes is specified
        if (nbytes != NULL) {
            cl_double io_rate_MBs = h_get_io_rate_MBs(
                elapsed, 
                *nbytes
            );
            std::printf(" (%.2f MB/s)", io_rate_MBs);
        }
        std::printf("\n");
    }
    
    return elapsed;
}
```

Every command submitted to a command queue may have an event associated with it. We can use this method to extract the time taken for work to complete.

### Instrumenting the buffer copy

We construct a **cl_event** object and use that event to collect timing information. For example, during writes to a device buffer we pass in the event to collect timing information.

```C++
    // mat_mult.cpp source
   cl_event io_event;

    H_ERRCHK(
        clEnqueueWriteBuffer(
            command_queue,
            A_d,
            blocking,
            0,
            nbytes_A,
            A_h,
            0,
            NULL,
            &io_event
        ) 
    );

```

Then, we use **h_get_event_time_ms** to extract the elapsed time and print out the transfer rate.

```C++
    // Time how long it takes to complete event
    cl_double upload_A_ms = h_get_event_time_ms(
        &io_event, 
        "Uploading Buffer A",
        &nbytes_A
    );
```

The buffer copies from **B_h** to **B_d**, and then from **C_d** to **C_h** are also instrumented in a similar way. From the previous call to **h_get_event_time_ms** we know **io_event** is in a complete state, so we can reuse it.

### Instrumenting the kernel

The **[clEnqueueNDRangeKernel](https://www.khronos.org/registry/OpenCL/sdk/3.0/docs/man/html/clEnqueueNDRangeKernel.html)** command accepts a pointer to an OpenCL Event. This event then tracks the time taken for the kernel to complete. 

```C++
    // Event for the kernel
    cl_event kernel_event;
    
    // Now enqueue the kernel
    H_ERRCHK(
        clEnqueueNDRangeKernel(
            command_queue,
            kernel,
            work_dim,
            NULL,
            global_size,
            local_size,
            0,
            NULL,
            &kernel_event
        ) 
    );

    // Time how long it takes to complete event
    cl_double run_kernel_ms = h_get_event_time_ms(
        &kernel_event, 
        "Running kernel",
        NULL
    );
```

In this manner we instrument the uploads, downloads, and kernel execution in the source file [mat_mult_profiling.cpp](mat_mult_profiling.cpp). Now we run the instrumented code and print out the results.

## Compile and run the appliciation

The makefile is set to compile the instrumented example [mat_mult_profiling.cpp](mat_mult_profiling.cpp). The program creates and fills matrices **A_d** and **B_d** with random numbers in the range [0-1] and then uses OpenCL to compute the solution in matrix **C_d**. The matrices are written to the following files in binary format:

* arrayA.dat
* arrayB.dat
* arrayC.dat

On your terminal change directory to **L5_Profiling** and compile and run with these commands (without the exclamation mark !).

In [1]:
!make clean; make mat_mult_profiling.exe; ./mat_mult_profiling.exe

rm -rf *.exe
CC -g -fopenmp -O2 -I../include mat_mult_profiling.cpp -o mat_mult_profiling.exe -lOpenCL
	               name: gfx1035 
	     Device version: OpenCL 2.0  
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
Time for event "Uploading Buffer A": 0.020 ms (26787.40 MB/s)
Time for event "Uploading Buffer B": 0.044 ms (24126.02 MB/s)
Time for event "Running kernel": 1.448 ms
Time for event "Downloading Buffer C": 0.074 ms (28828.36 MB/s)
Maximum error (infinity norm) is: 2.28882e-05


## Open-source profiling tools

### Tau

[Tau](https://www.cs.uoregon.edu/research/tau/home.php) is a commonly used open-source profiling and tracing toolkit for HPC applications. For OpenCL applications it provides both profiling and tracing functionality.

#### Profiling

The Tau application **tau_exec** can be used to collect profiling information. Profiling information can then be visualised with the Tau applications **paraprof** (GUI), or **pprof** (command-line).

We set the environment variables **PROFILEDIR=./tau** to tell Tau where to put files.

```bash
export PROFILEDIR=./tau
```

In [3]:
%env PROFILEDIR=./tau

env: PROFILEDIR=./tau


Then we use the following call to **tau_exec** to collect profiling information for opencl calls. The **-serial** flag is for non-mpi applications, and the **-opencl** flag instructs tau to collect on OpenCL calls.

In [12]:
!tau_exec -T serial -opencl ./mat_mult_profiling.exe -gpu

	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
device id: -2094813872.
command id: 94585970317664.
vendor id: 0.
Got a bogus start! 2 .TAU application
Time for event "Uploading Buffer A": 0.031 ms (17391.65 MB/s)
Time for event "Uploading Buffer B": 0.056 ms (19011.75 MB/s)
Time for event "Running kernel": 2.296 ms
Time for event "Downloading Buffer C": 0.076 ms (28385.02 MB/s)
Maximum error (infinity norm) is: 2.28882e-05


Now have a look at the contents of the tau directory

In [13]:
!ls ./tau

events.0.edf   profile.0.0.2  tautrace.0.0.0.trc  tau.trc
profile.0.0.0  profile.txt    tautrace.0.0.1.trc  trace.json
profile.0.0.1  tau.edf	      tautrace.0.0.2.trc


Use the Tau application **pprof** to get a text mode profile of the app.

In [11]:
!pprof > ./tau/profile.txt

We see from the profile that the call to **mat_mult** took approximately **12ms**. This is similar to what was measured from the profiling interface.

#### Tracing with Perfetto

For tracing we set the environment variables **TRACEDIR=./tau** **TAU_TRACE=1**.

```bash
export TAU_TRACE=1
export TRACEDIR=./tau
```

In [5]:
%env TAU_TRACE=1
%env TRACEDIR=./tau

env: TAU_TRACE=1
env: TRACEDIR=./tau


Capture OpenCL information as before

In [14]:
!tau_exec -T serial -opencl ./mat_mult_profiling.exe

	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
device id: 1324989776.
command id: 93919375223136.
vendor id: 0.
Got a bogus start! 2 .TAU application
Time for event "Uploading Buffer A": 0.031 ms (17122.09 MB/s)
Time for event "Uploading Buffer B": 0.064 ms (16409.19 MB/s)
Time for event "Running kernel": 2.287 ms
Time for event "Downloading Buffer C": 0.078 ms (27552.85 MB/s)
Maximum error (infinity norm) is: 2.28882e-05


Now merge the trace into a downloadable JSON document

In [7]:
!cd tau; echo 'y' | tau_treemerge.pl

/opt/tau/2.31.1/x86_64/bin/tau_merge -m tau.edf -e events.0.edf events.0.edf events.0.edf tautrace.0.0.0.trc tautrace.0.0.1.trc tautrace.0.0.2.trc tau.trc
tau.trc exists; override [y]? tautrace.0.0.0.trc: 450 records read.
tautrace.0.0.1.trc: 6 records read.
tautrace.0.0.2.trc: 41 records read.


In [15]:
!cd tau; tau_trace2json ./tau.trc ./tau.edf -chrome -ignoreatomic -o trace.json

Download the file **./tau/trace.json** to your computer. Then in your browser you can go to the address [https://ui.perfetto.dev](https://ui.perfetto.dev) and load the trace for viewing on your local machine. 


<figure style="margin-left:0; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="../images/Chrome_trace.png">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Tracing OpenCL calls with <a href="https://ui.perfetto.dev">ui.perfetto.dev</a>.</figcaption>
</figure>

In this instance we see that the kernel **mat_mult** has taken approximately 0.45ms to complete.

## Commercial profiling tools

### CLTracer

[CLTracer](https://www.cltracer.com/) is a commerical product that profiles OpenCL calls for Windows and Linux OpenCL applications. It requires a GUI to run and provides 

* A timeline of OpenCL calls, separated into API and kernel calls
* Tables of time spent in each call
    * Global and local kernel size recorded
    * Size of transfers recorded
* Time spent in the API vs time spent blocking
* Breakdown of time spent in kernels
* Breakdown of time spent in queues

Setting up project settings and running a trace for [mat_mult_profiling.cpp](mat_mult_profiling.cpp) was really easy. Unfortunately I don't see the ability to fetch information from the command line.

#### Timeline view

The timeline view shows when and for how long each OpenCL call lasts. The overview shows that creating the command queues was the most time-consuming operation in the application.

<figure style="margin-left:auto; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="../images/cltracer_timeline_overview.png">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">CLTracer timeline overview.</figcaption>
</figure>

If we zoom in to the kernel region we see that executing the kernel only took around 2.9ms and that it takes less time to upload and download arrays than the kernel spends executing.

<figure style="margin-left:auto; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="../images/cltracer_timeline_zoom.png">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">CLTracer timeline, zoomed in on kernel region.</figcaption>
</figure>

#### Tables

The timeline can also be viewed in tabular format. In addition to times for each OpenCL call you can see global and local sizes of the kernels as well as the size of kernel uploads and downloads.

<figure style="margin-left:auto; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="../images/cltracer_tables.png">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">CLTracer table of OpenCL calls.</figcaption>
</figure>

For more information on available tools within CLTracer please see the CLTracer [Documentation](https://www.cltracer.com/docs).

## Vendor profiling tools

### AMD

#### HSA application traces

The AMD utility **rocprof** has the ability to collect traces. We use the **--hsa-trace** option to collect statistics from the underlying HSA calls that provide the OpenCL implementation on AMD.

```bash
rocprof --hsa-trace -o rocprof_trace/result.csv ./mat_mult_profiling.exe -gpu
```

Inside the **rocprof_trace** folder you will find the following files:

| file | purpose |
| --- | --- |
| result.sysinfo.txt | System information on available devices |
| result.copy_stats.csv | Statistics on all IO calls |
| result.hsa_stats.csv | Statistics on HSA function calls |
| result.stats.csv | Statistics on all kernel calls |
| result.db | SQLITE3 database of profiling information |
| result.json | Trace information in JSON format |
| result.csv | Information on kernels such as **mat_mult** |

We can load the trace file **result.json** using Perfetto, available at [https://ui.perfetto.dev/](https://ui.perfetto.dev/). Copy the trace file **rocprof_trace/result.json** back to your computer and open it with the Perfetto UI. 

If you zoom (using the `wasd` keys) in you can see calls in GPU threads, COPY threads and HOST threads on the CPU. If you click on the **mat_mult.kd** function you can see how long the kernel took to execute.

<figure style="margin-left:0; margin-right:auto; width:100%;">
    <img style="vertical-align:middle" src="../images/Perfetto_UI_kernel.png">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Determining the time for a kernel call</figcaption>
</figure>

This is a useful method to see how long a kernel execution took, however other OpenCL calls are not directly visible.

#### Hardware performance counters with rocprof

Hardware performance counters are devices in a processor that measure events, such as the number of wavefronts executed, or the number of times a cache is missed. Rocprof can collect performance counters on kernels. The type of performance counter information that can be captured is obtained with this command:

```bash
rocprof --list-derived
```

We can specify the counters to collect in a file such as [rocprof_counters.txt](rocprof_counters.txt). Here we specify some commonly used metrics for collection. Each **pmc** line is a unique experiment involving an individual run of the code. In this example we collect stats for the **mat_mult** kernel for the first 64 work-items on GPU 0.

```txt
# Cache hits and Cache misses
pmc: TCC_HIT_sum, TCC_MISS_sum

# Total video memory fetched and written
pmc: FETCH_SIZE, WRITE_SIZE

# Percentage of time the GPU was busy, total wavefronts executed
pmc: GPUBusy, Wavefronts

# Average number of vector and scalar instructions executed per work-item
pmc: VALUInsts, SALUInsts

# Average number of vector and scalar fetch instructions per work-item
pmc: VFetchInsts, SFetchInsts

# Average number of vector write instructions per work-item
pmc: VWriteInsts

# Average number of shared and global memory read or write instructions per work item
pmc: LDSInsts, GDSInsts

# Percentage of active vector ALU threads in a wave, percentage of GPU time vector and scalar instructions are processed
pmc: VALUUtilization, VALUBusy, SALUBusy, 

# Percentage of fetch, write, atomic, and other instructions that hit the L2 cache
pmc: L2CacheHit

# Percentage of time the memory unit is active (including stalled), and just stalled, percentage of time the write unit is stalled
pmc: MemUnitBusy, MemUnitStalled, WriteUnitStalled

# Percentage of time ALU's are stalled by shared memory access, percentage of GPU time local memory is stalled by bank conflicts
pmc: ALUStalledByLDS, LDSBankConflict

# Dispatches range, which work-items to profile
range: 0 : 64
# Which GPU's to profile
gpu: 0
# Names of kernels to profile
kernel: mat_mult
```

Then we can use rocprof to collect the data for these counters.

```bash
rocprof -i rocprof_counters.txt --timestamp on --stats -o rocprof_counters/result.csv ./mat_mult_profiling.exe -gpu
```

If your chosen performance counters are supported, then the file [rocprof_counters/result.csv](rocprof_counters/result.csv) should contain a count for every time the counter was triggered. The file [rocprof_counters/example.csv](rocprof_counters/example.csv) is an example file collected with rocprof on **mat_mult_profiling.exe**. This [page](https://docs.amd.com/bundle/ROCProfiler-User-Guide-v5.1/page/rocprof_Command_Line_Tool.html) has information on what the keys in the CSV file mean.

#### Rocprof under a job manager

Rocprof runs fine under a job manager like SLURM, you just need to make an output file for each process launched. For example on SLURM the `$SLURM_JOBID` and `$SLURM_PROCID` environment variables are helpful in separating the output. Put the rocprof commands in a script called **profile.sh**.

```bash
#!/bin/bash
rocprof -i rocprof_counters.txt -o rocprof_counters/result-$SLURM_JOBID-$SLURM_PROCID.csv ./mat_mult_profiling_mpi.exe
```

Then you can run the script from **srun** like this so it picks up the environment variable **$SLURM_PROCID** from within the script.

```bash
srun -N $SLURM_JOB_NUM_NODES -n $SLURM_NTASKS -c $OMP_NUM_THREADS ./profile.sh
```

A complete example for using rocprof with an MPI-enabled application is in **course_material/L2_Using_OpenCL_On_Setonix/rocprof_mpi**.

### NVIDIA

Historically there was limited functionality for profiling OpenCL events with NVIDIA's [NVVP](http://uob-hpc.github.io/2015/05/27/nvvp-import-opencl.html), however profiling support for OpenCL has largely disappeared, with the implementation of NSight Compute and NSight Systems.




<address>
Written by Dr. Toby Potter of <a href="https://www.pelagos-consulting.com">Pelagos Consulting and Education</a> for the <a href="https://pawsey.org.au">Pawsey Supercomputing Research Centre</a>. All trademarks mentioned are the property of their respective owners.
</address>